# Задание F

Составить (с использованием любого модуля морфологического анализа) программу, выполняющую синтаксическую сегментацию русскоязычного текста на базе нескольких выбранных правил:
* сегментацию на простые предложения по знакам пунктуации 		и/или
* выделение неразрывных синтаксически связанных групп слов на основе локальных высоковероятных связей (примеры правил есть на слайдах 34, 50, 51 презентации по СА), например: синий платок, очень красивый (или даже более сложные группы: исключительно интересный фильм, красный полосатый мяч, любит весело играть, которые выделяются применением нескольких правил).

Протестировать программу на нескольких фрагментах текста (не менее 1-2 страниц).

Отчет: описание применяемых правил сегментации, результаты тестов, программа и комментарий к ней.

# Предисловие

Сразу отметим, что в наши цели не входит написание идеальной сегментации (которая работает для любых текстов и учитывает различные "исключительные" случаи), идеального синтаксического анализатора (который работает для любых предложений и строит правильные синтаксические деревья) и так далее. В первую очередь, это связано с самим заданием, предполагающим следование некоторым правилам, как-то:
* сегментация по знакам пунктуации -- простейшая сегментация не учитывает сокращения вида "А. С. Пушкин", некорректно обратывает предложение с прямой речью в нем и проч.;
* выделение синтаксически связанных групп слов на основе локальных высоковероятных связей -- набор правил, данный в презентациях (и может даже несколько расширенный) не дает возможности построить корректное дерево зависимостей для произвольного предложения (помимо того, что дерево зависимостей обладает своими недостатками, применение этих правил также не дает гарантированно верного результата).

Поэтому обговорим задачи, которые были поставлены и выполнены в этом задании:
* сколько-нибудь качественная сегментация литературных текстов, следующая принципам "наивной" графематики с некототорыми уточнениями, связанными с форматом текстов, нами взятых;
* реализация правил, данных в презентации, с некоторыми уточнениями, связанными с работой используемого морфоанализатора;
* выделение синтаксически связанных групп слов путем синтаксического анализа каждого предложения, выполняющегося путем проверки наличия связи каждого слова с его "соседями" (с уточнением того, что стоит понимать под "соседними" словами);
* и, наконец, описание недостатков построенных моделей.

# Часть 1. Сегментация текста на предложения.

Нами будут рассматриваться тексты, взятые с сайта lib.ru: они были банально скопированы в текстовый файл во избежание работы с html-файлами. В рассматриваемых текстах есть членение текста на абзацы, которое длостаточно просто выделяется, также в предложениях присутствуют символы '\n', которые реализуют перенос строк.

Создадим класс <b>Split</b>, который реализует сегментацию полученных текстов (заданных по имени файлов) на:
* абзацы - путем выделения "красных" строк;
* предложения - путем разбиения по терминальным знакам пунктуации (.!? в различных их комбинациях);
* "подпредложения" - путем разбиения по знакам пунктуации (.!?:,), в дальнейшем использоваться не будет.

Стоит отметить, что написанные типы сегментации могут применяться независимо друг от друга, это влияет лишь на полученную структуру текста: при каждой сегментации каждому полученному объекту, будь то абзац, предложение или "подпредложение", сопоставляется свой порядковый индекс; каждая следующая сегментация применяется для всех объектов полученной структуры.

In [1]:
import os, re
# texts -> paragraphs -> sentences -> words -> morph_parse

# texts open
open_text = lambda name: open(os.path.join(os.getcwd(), name), 'r').read()

class Split:
    def __init__(self, *names):
        if len(names) == 0:
            self.texts = None
        elif len(names) == 1:
            self.texts = {() : open_text(*names)}
        else:
            self.texts = {(i,) : text_i for i, text_i in enumerate(map(open_text, names))}
        
        # paragraph segmentation function
        self.par_split = lambda text: re.split('[.!? ]*\n+ {3} *', text)
        self.par_segment = lambda: self.segment(self.texts, self.par_split)
        # sentence segmentation function
        self.sent_split = lambda text: re.split('[.!?]+ *', re.sub('\n\n+', '.', text))
        self.sent_segment = lambda: self.segment(self.texts, self.sent_split)
        # subsentence segmentation function
        self.subsent_split = lambda text: re.split('[.!?:,]+ *', re.sub('\n\n+', '.', text))
        self.subsent_segment = lambda: self.segment(self.texts, self.subsent_split)
    
    # segmentation function - segments all objects in self.texts
    def segment(self, text, split_func):
        self.texts = {key + (i,) : value_i for key, value in text.items()
                     for i, value_i in enumerate(split_func(value))}
        return sorted(self.texts.items())
    
    def __repr__(self):
        return str(self.texts)
    def __getitem__(self, key):
        result = sorted([(i, t_i) for i, t_i in self.texts.items() if i[:len(key)] == key])
        return result[0][1] if len(result) == 1 else result
    def __iter__(self):
        return iter(sorted(self.texts.items()))

In [2]:
split = Split('text_1.txt', 'text_2.txt')
split.par_segment()
split.sent_segment()
print_str = lambda items: '\n'.join(map(lambda item: '{} : {}'.format(*item), items))
print(print_str(split[(0,)][:10]), end = '\n{}\n'.format(100 * '.'))
print(print_str(split[(1,)][:10]), end = '\n{}\n'.format(100 * '.'))

(0, 0, 0) : Говард Ф
(0, 0, 1) : Лавкрафт
(0, 0, 2) : Музыка Эриха Цанна
(0, 1, 0) : Я  самым внимательным образом изучил карты города, но так и  не отыскал
на  них  улицу  д'Осейль
(0, 1, 1) : Надо сказать, что  я  рылся отнюдь  не  только  в
современных  картах,  поскольку мне  было  известно, что  подобные  названия
нередко меняются
(0, 1, 2) : Напротив, я, можно сказать,  по уши залез  в седую старину
и, более того, лично обследовал интересовавший меня район, уже  не  особенно
обращая  внимания на  таблички  и  вывески,  в  поисках  того,  что  хотя бы
отдаленно походило на интересовавшую  меня улицу д'Осейль
(0, 1, 3) : Однако,  несмотря
на  все мои усилия, вынужден сейчас не без  стыда признаться,  что  так и не
смог отыскать нужные мне дом, улицу, и даже приблизительно определить район,
где,  в  течение  последних месяцев  моей  обездоленной  жизни,  я,  студент
факультета метафизики, слушал музыку Эриха Занна
(0, 2, 0) : Меня отнюдь не  удивляет  подобный провал в памяти, поско

В приведенном выше примере мы можем видеть работу данного класса: мы подали на вход два текста, каждый из которых сначала разбили на абзацы, а затем на предложения. В результате мы получили список предложений с номерами вида (x, y, z), где x - номер текста, y - номер абзаца, z - номер предложения.

Для удобства обращения с полученной структурой был переопределен метод __getitem__: по ключу он возвращает все объекты структуры, номера которых своим началом совпадают с этим ключом. Это позволяет работать с конкретным текстом, абзацем конкретного текста или предложением (по номерам вида (a,), (a, b), (a, b, c) соответственно).

Можно видеть недостаток "наивной" графематики: так, ее использование приводит к разделению на два предложения словосочетания "Говард Ф.Лавкрафт". В взятых нами текстах практически отсутствуют сокращения, поэтому мы будем довольствоваться и такой сегментацией.

# Часть 2. Определение синтаксической связи двух слов.

Для определения синтаксической связи двух слов мы будем следовать определенным правилам высоковероятных связей, а значит нам потребуется использование морфоанализатора (мы не собираемся обрабатывать размеченный корпус текстов для автоматического определения правил, как не делали это и в случае с сегментацией).

Для наших целей вполне подходит pymorphy2.MorphAnalyzer(): его метод parse() сопоставляет слову его возможные разборы; наибольший интерес представляют:
* грамматические "теги" - набор грамем, - каждого разбора: именно они необходимы для применения каждого из правил;
* нормальная форма слова: необходима для определения принадлежности к какой-либо группе (подробнее об этом будет сказано позже);
* параметр <i>score</i>: мы будем стремиться к использованию информации обо всех возможных разборах, но тем не менее будем использовать его для оценки вероятности выполнения конкретного правила (то есть для снятия омонимии словосочетания - подробнее об этом будет сказано позже).

In [3]:
import pymorphy2
# useful links:
# https://pymorphy2.readthedocs.io/en/latest/user/guide.html
# https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html#russian-genders
# http://opencorpora.org/dict.php?act=gram
help(pymorphy2.MorphAnalyzer().TagClass)

Help on class OpencorporaTag in module pymorphy2.tagset:

class OpencorporaTag(builtins.object)
 |  Wrapper class for OpenCorpora.org tags.
 |  
 |  .. warning::
 |  
 |      In order to work properly, the class has to be globally
 |      initialized with actual grammemes (using _init_grammemes method).
 |  
 |      Pymorphy2 initializes it when loading a dictionary;
 |      it may be not a good idea to use this class directly.
 |      If possible, use ``morph_analyzer.TagClass`` instead.
 |  
 |  Example::
 |  
 |      >>> from pymorphy2 import MorphAnalyzer
 |      >>> morph = MorphAnalyzer()
 |      >>> Tag = morph.TagClass  # get an initialzed Tag class
 |      >>> tag = Tag('VERB,perf,tran plur,impr,excl')
 |      >>> tag
 |      OpencorporaTag('VERB,perf,tran plur,impr,excl')
 |  
 |  Tag instances have attributes for accessing grammemes::
 |  
 |      >>> print(tag.POS)
 |      VERB
 |      >>> print(tag.number)
 |      plur
 |      >>> print(tag.case)
 |      None
 |  
 |  Avai

Для дальнейшей работы необходимо разобраться в системе тегов, реализованных классом pymorphy2.MorphAnalyzer().TagClass на основе тегов OpenCorpora и отличия от них.

Одно из отличий, например, прописано в документации:  
" В OpenCorpora (на июль 2013) есть еще падежи gen1 и loc1.
Они указываются вместо gent/loct, когда у слова есть форма gen2/loc2. 
В pymorphy2 gen1 и loc1 заменены на gent/loct, чтоб с ними было проще работать. "

В целом, нас будут интересовать не отличия от OpenCorpora, а особенности разбора слов с использованием pymorphy2.MorphAnalyzer().

Ниже дан список тегов, которые могут быть нам полезны, с их возможными значениями с их кирилличесским вариантом для большей ясности.

In [4]:
morph_analyzer = pymorphy2.MorphAnalyzer()
meaningful_tags = {'ANIMACY' : '.animacy(), одушевленность',
                   'ASPECTS' : '.aspect(), вид (совершенный или несовершенный)',
                   'CASES' : '.case(), падеж',
                   'GENDERS' : '.gender(), род (мужской, женский, средний)',
                   'INVOLVEMENT' : '.involvement(), включенность говорящего в действие',
                   'MOODS' : '.mood(), наклонение (повелительное, изъявительное)',
                   'NUMBERS' : '.number(), число (единственное, множественное)',
                   'PARTS_OF_SPEECH' : '.POS(), часть речи',
                   'PERSONS' : '.person(), лицо (1, 2, 3)',
                   'TENSES' : '.tense(), время (настоящее, прошедшее, будущее)',
                   'TRANSITIVITY' : '.transitivity(), переходность (переходный, непереходный)',
                   'VOICES' : 'voice, залог (действительный, страдательный)'}
pos_lat_tags = lambda tag: list(morph_analyzer.TagClass.__dict__[tag])
pos_cyr_tags = lambda tag: [morph_analyzer.lat2cyr(t) for t in pos_lat_tags(tag)]
print('\n\n'.join(['{} - {}:\n{}'.format(tag, desc, list(zip(pos_lat_tags(tag), pos_cyr_tags(tag))))
                   for tag, desc in meaningful_tags.items()]))

ANIMACY - .animacy(), одушевленность:
[('inan', 'неод'), ('anim', 'од')]

INVOLVEMENT - .involvement(), включенность говорящего в действие:
[('incl', 'вкл'), ('excl', 'выкл')]

NUMBERS - .number(), число (единственное, множественное):
[('plur', 'мн'), ('sing', 'ед')]

CASES - .case(), падеж:
[('loc2', 'пр2'), ('loc1', 'пр1'), ('nomn', 'им'), ('acc2', 'вн2'), ('accs', 'вн'), ('voct', 'зв'), ('gent', 'рд'), ('ablt', 'тв'), ('gen2', 'рд2'), ('gen1', 'рд1'), ('loct', 'пр'), ('datv', 'дт')]

TENSES - .tense(), время (настоящее, прошедшее, будущее):
[('pres', 'наст'), ('futr', 'буд'), ('past', 'прош')]

PARTS_OF_SPEECH - .POS(), часть речи:
[('PRTF', 'ПРИЧ'), ('INFN', 'ИНФ'), ('ADVB', 'Н'), ('VERB', 'ГЛ'), ('NOUN', 'СУЩ'), ('GRND', 'ДЕЕПР'), ('NUMR', 'ЧИСЛ'), ('PRED', 'ПРЕДК'), ('ADJS', 'КР_ПРИЛ'), ('PRTS', 'КР_ПРИЧ'), ('COMP', 'КОМП'), ('INTJ', 'МЕЖД'), ('PREP', 'ПР'), ('NPRO', 'МС'), ('CONJ', 'СОЮЗ'), ('ADJF', 'ПРИЛ'), ('PRCL', 'ЧАСТ')]

MOODS - .mood(), наклонение (повелительное, изъявите

Итак, как уже было сказано выше, реализация правил сопряжена с пониманием особенностью разбора каждого из слов в анализируемой паре.  
Ниже будут продемонстрированы некоторые из ключевых особенностей, которыми мы будем пользоваться (или будем с ними мириться).

### Список ключевых (для нашей задачи) особенностей pymorphy2.MorphAnalyzer()

#### Неизменяемым существительным и прилагательными сопоставляются разборы со всеми возможные падежи.

In [5]:
word_tags = lambda word, tag: "'{}':\n{}".format(word, ', '.join({str(getattr(var.tag, tag))
                                                                  for var in morph_analyzer.parse(word)}))
print('падежи неизменяемого существительного ' + word_tags('пальто', 'case'), end = '\n\n')
print('падежи неизменяемого прилагательного ' + word_tags('электрик', 'case'), end = '\n\n')

падежи неизменяемого существительного 'пальто':
nomn, accs, gent, ablt, loct, datv

падежи неизменяемого прилагательного 'электрик':
nomn, accs, gent, ablt, loct, datv



Мы будем пользоваться этим при определении согласования: отпадает необходимость проверять "неизменяемость" слов.

#### Прилагательному во множественной форме в разборах не сопоставляется род (в отличие от существительных).

In [6]:
word_parse = lambda word: "'{}':\n{}".format(word, '\n'.join(map(lambda p: '{} + ({})'.format(p.normal_form, p.tag),
                                                                 morph_analyzer.parse(word))))
print('прилагательное во множественной форме ' + word_parse('неприятные'), end = '\n\n')
print('существительное во множественной форме ' + word_parse('проблемы'), end = '\n\n')

прилагательное во множественной форме 'неприятные':
неприятный + (ADJF,Qual plur,nomn)
неприятный + (ADJF,Qual inan,plur,accs)

существительное во множественной форме 'проблемы':
проблема + (NOUN,inan,femn sing,gent)
проблема + (NOUN,inan,femn plur,nomn)
проблема + (NOUN,inan,femn plur,accs)



Об этом также важно помнить при определении согласования: если в единственном числе мы требуем совпадения по падежу, числу и роду (case, number & gender), то во множественном совпадение должно быть только по числу и падежу.

#### Отглагольному прилагательному сопоставляется разбор, в котором оно является причастием (а нормальная форма - глагол).

In [7]:
print('отглагольное прилагательное ' + word_parse('открытый'), end = '\n\n')
# с отглагольным существительным такого не происходит
print('отглагольное существительное ' + word_parse('освобождение'), end = '\n\n')

отглагольное прилагательное 'открытый':
открытый + (ADJF,Qual masc,sing,nomn)
открытый + (ADJF,Qual inan,masc,sing,accs)
открыть + (PRTF,perf,tran,past,pssv masc,sing,nomn)
открыть + (PRTF,perf,tran,past,pssv inan,masc,sing,accs)

отглагольное существительное 'освобождение':
освобождение + (NOUN,inan,neut sing,nomn)
освобождение + (NOUN,inan,neut sing,accs)



Это свойство будет использоваться при определении отглагольности прилагательного; к сожалению, определить отглагольность существительного с помощью одного лишь морфоанализатора вызывает затруднения.

https://ru.wikipedia.org/wiki/Отглагольное_существительное  
https://ru.wiktionary.org/wiki/Категория:Русские_отглагольные_существительные  
https://ru.wiktionary.org/wiki/-ациj  
https://ru.wiktionary.org/wiki/-тель

Воспользовавшись данными ссылками можно узнать про самый популярный способ образования отглагольных существительных:
* при помощи суффикса -ни-е (-ани-е, -ени-е, -яни-е);
* при помощи суффикса -аци-я (при добавлении к основе глаголов с иноязычной основой);
* при помощи суффикса -тель (при добавлении к основе неопределённой формы глагола);

а также ознакомиться с другими способами их образования. Можно видеть, что приведенные способы образования дают наибольшее количество примеров подобных существительных: этот факт и можно было бы использовать в качестве правила, а для прочих "редких" форм образования составить словарь отглагольных существительных.

Но, конечно же, есть исключения:
* зрение, сознание (происходят от глагола, но потеряло свою отглагольность), поколение (от праслав. *kolěno, корень -колен-)
* метель (происходит от глагола, но потеряло свою отглагольность), картель (от фр. cartel — «вызов на поединок»)
* нация (от лат. natio - племя, народ)

Поэтому составленное по описанному выше принципу правило будет работать в большом количестве случаев, но не гарантирует верного результата. Тем не менее, нами будет использоваться именно такое правило, с добавлением словаря "редких" отглагольных существительных.

#### Глаголу в нормальной форме (то есть инфинитиву) сопоставляется отличная от VERB часть речи - а именно INFΝ.

In [8]:
print('глагол ' + word_parse('играю'), end = '\n\n')
print('глагол в нормальной форме (инфинитив) ' + word_parse('играть'), end = '\n\n')

глагол 'играю':
играть + (VERB,impf,tran sing,1per,pres,indc)

глагол в нормальной форме (инфинитив) 'играть':
играть + (INFN,impf,tran)



Это необходимо учитывать в формулировке правил, в которых участвует глагол в качестве главного слова и связь сохраняется при постановке его в нормальную форму.

### Такие разные местоимения.

Посмотрим, какие разборы сопоставляются местоимениям:

In [9]:
print('личное местоимение ' + word_parse('нас'), end = '\n\n')
print('относительное местоимение ' + word_parse('кого'), end = '\n')
print('относительное местоимение ' + word_parse('который'), end = '\n\n')
print('указательное местоимение ' + word_parse('стольких'), end = '\n')
print('указательное местоимение ' + word_parse('таков'), end = '\n\n')
print('определительное местоимение ' + word_parse('всякому'), end = '\n\n')
print('отрицательное местоимение ' + word_parse('никому'), end = '\n')
print('отрицательное местоимение ' + word_parse('ничьему'), end = '\n\n')
print('неопределенное местоимение ' + word_parse('несколько'), end = '\n')
print('неопределенное местоимение ' + word_parse('кое-кто'), end = '\n\n')

личное местоимение 'нас':
мы + (NPRO,1per plur,gent)
мы + (NPRO,1per plur,accs)
мы + (NPRO,1per plur,loct)

относительное местоимение 'кого':
кто + (NPRO,masc sing,accs)
кто + (NPRO,masc sing,gent)
относительное местоимение 'который':
который + (ADJF,Apro,Subx,Anph masc,sing,nomn)
который + (ADJF,Apro,Subx,Anph inan,masc,sing,accs)

указательное местоимение 'стольких':
столько + (NUMR gent)
столько + (NUMR anim,accs)
столько + (NUMR loct)
указательное местоимение 'таков':
таков + (ADJS,Apro masc,sing)

определительное местоимение 'всякому':
всякий + (ADJF,Apro masc,sing,datv)
всякий + (ADJF,Apro neut,sing,datv)
всякий + (NPRO,masc sing,datv)
всякое + (NPRO,neut sing,datv)

отрицательное местоимение 'никому':
никто + (NPRO sing,datv)
отрицательное местоимение 'ничьему':
ничей + (ADJF,Apro masc,sing,datv)
ничей + (ADJF,Apro neut,sing,datv)

неопределенное местоимение 'несколько':
несколько + (ADVB)
несколько + (NUMR nomn)
несколько + (NUMR inan,accs)
неопределенное местоимение 'кое-кто':

Можно видеть, что далеко не всем местоимениям сопоставляется часть речи 'NPRO'; тем не менее, всем местоимениям, что изменяются по родам, числам и падежам и согласуются с существительным, сопоставлены их число, падеж и род; в случае склонения только по падежам в разборах прописан падеж. Из-за таких разных и множественных интерпретаций трудно добавить обработку местоимений - тем более, некоторые из них (те, что имеют в разборах часть речи, которой они могут соответствовать).

Отметим лишь тот факт, что морфоанализатор может ставить местоимениям в качестве части речи: NPRO, ADJF, ADJS, NUMR, но никак не NOUN. 

### Ухищрения в дополнение к использованию морфоанализатора

In [10]:
print(word_parse('среди'))
print(word_parse('между'))

'среди':
среди + (PREP)
'между':
между + (PREP)


Ранее мы пришли к выводу, что нам необходима дополнительная информация об отглагольных существительных. То же самое верно и для предлогов, поэтому нам необходим словарь соответствий предлогов и обслуживаемых ими падежей.

Поэтому мы воспользовались ссылкой https://ru.wiktionary.org/wiki/Категория:Русские_предлоги и составили словарь предлогов, употребляющихся с родительным, дательным, винительным, творительным и предложным падежами

Для того, чтобы понять, какие предлоги обслуживают второй родительный, второй винительный и второй предложный, мы воспользовались ссылками http://rusgram.ru/Падеж и http://rusgram.ru/Предложный_падеж#12 , где говорится следующее:
* ...Второй винительный падеж (другие названия – включительный, превратительный, собирательный) встречается после предлога в при небольшом количестве глаголов...
*  ...У некоторых существительных после локативных предлогов в и на употребляется особая форма предложного падежа...  
...локативная форма употребляется не со всеми предлогами, управляющими предложным падежом: встречается после в и на, но не встречается после о, при, по;

Для второго родительного падежа списка употребляемых с ним предлогов не дано, но даны примеры словосочетаний с ним:
* ложка сахару; чашка чаю; Народу набежало!; Шуму было!
* ни разу; выпьем чайку;
* без году неделя; нашего полку прибыло; с миру по нитке; моя хата с краю; беситься с жиру и др.

В тексте статьи говорится, что хоть второй родительный широко используется в разговорной речи, но обязательным является лишь в отдельных случаях, которые по большей части являются фразеологизмами. Из этого нами был сделан вывод, что будет достаточно добавить в предлоги, которые обслуживают второй родительный падеж, только <i>без</i> и <i>с</i>.

In [11]:
for preps_str in re.split('\n\_+[\w\-]*\_+\n', open_text('preps&cases.txt'))[1:]:
    print((lambda p: '{} :\n{}'.format(p[0], p[1:]))(re.split('\n', preps_str)))

gent :
['без', 'без ведома', 'безо', 'близ', 'близко от', 'в виде', 'в зависимости от', 'в качестве', 'в лице', 'в отличие от', 'в отношении', 'в пандан', 'в преддверии', 'в продолжение', 'в результате', 'в роли', 'в силу', 'в течение', 'в целях', 'вблизи', 'ввиду', 'вглубь', 'вдоль', 'взамен', 'вместо', 'вне', 'внизу', 'внутри', 'внутрь', 'во благо', 'вовнутрь', 'возле', 'вокруг', 'впереди', 'впредь до', 'вроде', 'вследствие', 'для', 'до', 'за вычетом', 'за исключением', 'за счёт', 'заместо', 'из', 'из-за', 'из-под', 'изнутри', 'изо', 'исходя из', 'кроме', 'кругом', 'меж', 'между', 'мимо', 'на благо', 'на виду у', 'на глазах у', 'на предмет', 'наверху', 'накануне', 'наподобие', 'напротив', 'насупротив', 'насчёт', 'начиная с', 'не без', 'не считая', 'недалеко от', 'независимо от', 'ниже', 'обок', 'около', 'окромя', 'округ', 'от', 'от имени', 'от лица', 'относительно', 'ото', 'по линии', 'по мере', 'по поводу', 'по причине', 'по случаю', 'поблизости от', 'поверх', 'под видом', 'под эгид

Создадим класс <b>BigramParser</b>, который реализует определение синтаксической связи двух слов при помощи метода connect, который по двум словам (порядок которых важен) возвращает список вида <b>[((главное слово, тип синтаксической связи), score)]</b>, упорядоченный по параметру <i>score</i>, где <i>score</i> - это "вероятность" данного разбора.

Для каждого типа связи есть правило, которое подразумевает определенный набор характеристик для каждого из слов: в общем случае одной из характеристик является часть речи первого и второго слова, и эти характеристики проверяются в первую очередь, но в ряде случаев этого недостаточно:
* согласование A|P -> N предполагает совпадающий число, падеж и род (в случае ед.ч.);
* зависимость Prep -> N предполагает соответствие падежа существительного падежам, обслуживаемым предлогом;
* зависимость N -> Prep предполагает отглагольность существительного;
* зависимость N -> N предполагает родительный падеж (или второй родительный, например: ложка сахару, чашка чаю) второго существительного;
* зависимость V -> N предполагает винительный падеж существительного (но не второй винительный, поскольку он "встречается после предлога в при небольшом количестве глаголов", например: [пойти, записаться, выбиться,...] в [солдаты,летчики,начальники])

Как можно видеть, большинство характеристик касаются варианта разбора одного из слов (помимо отглагольности существительного, которая определяется нами "вручную"). Мы устанавливаем связь между двумя словами, если в их морфологических разборах присутствует такие разборы, что они удовлетворяют заданным нами характеристикам. Для каждого из слов мы можем найти все подходящие под эти характеристики разборы и посчитать "вероятность соответствия" этим характеристикам - она будет определяться, как сумма соответсвующих этим разборам параметров <i>score</i>. Параметр же <i>score</i> для всей связи вычисляется как произведение полученных суммарных вероятностей, то есть как вероятность двух независимых событий: соответствия каждого из слов своим характеристикам.

Перечислим список реализованных вспомогательных методов:
* morph_parse: возвращает набор всех возможных морфологических разборов слова;
* normal_forms: возвращает набор всех возможных нормальных форм слова;
* tag_scores: возвращает словарь вида {тег разбора : его вероятность}
* check_tag: по слову и списку значений тегов возвращает наличие хотя бы одного из значений хотя бы в одном из разборов;
* morph_tags: по слову и списку тегов возвращает значения этих тегов для каждого разбора;
* dicts_init: загружает словари "редких" отглагольных существительных и предлогов для каждого из падежей + создает словарь вида {предлог : список обслуживаемых падежей}

Также в методе connect активно используется функция check_POSes, которая по двум наборам частей речи проверяет, не пусто ли пересечение каждого из наборов с возможными частями речи каждого из слов.

In [12]:
class BigramParser:
    def __init__(self):
        # morphological analyzer
        self.morph_analyzer = pymorphy2.MorphAnalyzer()
        
        self.morph_parse = lambda word: self.morph_analyzer.parse(word)
        
        self.normal_forms = lambda word: {var.normal_form for var in self.morph_parse(word)}
        
        self.tag_scores = lambda word: {var.tag : var.score for var in self.morph_parse(word)}
        
        self.check_tag = lambda word, *tag_vals: any(t in var for var in self.tag_scores(word) for t in tag_vals)
        
        self.morph_tags = lambda word, *tags: {','.join((str(getattr(var.tag, t)) for t in tags))
                                               for var in self.morph_parse(word)}
        
        self.dicts_init()
    
    def dicts_init(self, case_preps_txt = 'preps&cases.txt', verbal_nouns_txt = 'verbal_nouns.txt'):
        self.case_preps = dict((lambda p: (p[0], p[1:]))(re.split('\n', preps))
                                for preps in re.split('\n\_+[\w\-]*\_+\n', open_text(case_preps_txt))[1:])
        self.prep_cases = {p : {c for c in self.case_preps if p in self.case_preps[c]}
                            for c in self.case_preps for p in self.case_preps[c]}
        self.verbal_nouns = set(re.split('\n', open_text(verbal_nouns_txt)))
    
    def connect(self, first, second):
        POSes = dict(zip(('first', 'second'), map(lambda word: set(self.morph_tags(word, 'POS')), (first, second))))
        check_POSes = lambda p_1, p_2: set(p_1.split(','))&POSes['first'] and set(p_2.split(','))&POSes['second']
        
        pos_connects = {}
        score = lambda word, *pos_tags: sum(score for tag, score in self.tag_scores(word).items() for t in pos_tags
                                            if 'None' not in t and set(t.split(',')) in tag)
        
        if check_POSes('ADJF,NUMR,PRTF', 'NOUN'):
            POS_group = POSes['first']&{'ADJF', 'NUMR', 'PRTF'}
            sing_tag_group = {'case', 'number', 'gender'}
            plur_tag_group = {'case', 'number'}
            if POS_group:
                sing_tags = self.morph_tags(first, *sing_tag_group)&self.morph_tags(second, *sing_tag_group)
                plur_tags = self.morph_tags(first, *plur_tag_group)&self.morph_tags(second, *plur_tag_group)
                concord_tags = plur_tags if self.check_tag(first, 'plur') and self.check_tag(second, 'plur') else sing_tags
                # concord : {ADJF,NUMR,PRTF} <- NOUN
                if concord_tags:
                    first_score = score(first, *['{},{}'.format(POS, case) for POS in POS_group for case in concord_tags])
                    second_score = score(second, *['{},{}'.format('NOUN', case) for case in concord_tags])
                    pos_connects[(second, '{} <- NOUN'.format('|'.join(POS_group)))] = first_score*second_score
                # PRTF -> NOUN
                elif 'PRTF' in POSes['first']:
                    pos_connects[(first, 'PRTF -> NOUN')] = score(first, 'PRTF')*score(second, 'NOUN')

        # PREP -> NOUN
        if check_POSes('PREP', 'NOUN'):
            if first.lower() in self.prep_cases and self.prep_cases[first.lower()]&self.morph_tags(second, 'case'):
                pos_connects[(first, 'PREP -> NOUN')] = score(first, 'PREP')*score(second, 'NOUN')

        # NOUN -> PREP   
        if check_POSes('NOUN', 'PREP'):
            if first[-4:-1] in {'ани', 'ени', 'яни', 'аци', 'тел'} or self.normal_forms(first)&self.verbal_nouns:
                pos_connects[(first, 'NOUN -> PREP')] = score(first, 'NOUN')*score(second, 'PREP')

        # NOUN -> NOUN + gent (or gen2)
        if check_POSes('NOUN', 'NOUN') and self.check_tag(second, 'gent', 'gen2'):
            pos_connects[(first, 'NOUN -> NOUN')] = score(first, 'NOUN')*score(second, 'NOUN,gent', 'NOUN,gen2')
        
        # {VERB,INFN} -> NOUN + accs
        if check_POSes('VERB,INFN', 'NOUN') and self.check_tag(second, 'accs'):
            POS_group = POSes['first']&{'VERB', 'INFN'}
            first_score = score(first, *POS_group)
            second_score = score(second, 'NOUN,accs')
            pos_connects[(first, '{} -> NOUN'.format('|'.join(POS_group)))] = first_score*second_score

        # NUMR <- NOUN
        if check_POSes('NUMR', 'NOUN'):
            pos_connects[(second, 'NUMR <- NOUN')] = score(first, 'NUMR')*score(second, 'NOUN')

        # {VERB,ADJF,PRTF,NOUN} -> INFN
        if check_POSes('NOUN', 'INFN') and (first[-4:-1] in {'ани', 'ени', 'яни', 'аци', 'тел'} or
                                            self.normal_forms(first)&self.verbal_nouns):
            pos_connects[(first, 'NOUN -> INFN')] = score(first, 'NOUN')*score(second, 'INFN')
        if check_POSes('VERB,ADJF,PRTF', 'INFN'):
            POS_group = POSes['first']&{'VERB', 'ADJF', 'PRTF'}
            pos_connects[(first, '{} -> INFN'.format('|'.join(POS_group)))] = score(first, *POS_group)*score(second, 'INFN')

        # ADVB <- {ADVB,ADJF,PRTF,VERB,INFN}
        if check_POSes('ADVB', 'ADVB,ADJF,PRTF,VERB,INFN'):
            POS_group = POSes['second']&{'ADVB', 'ADJF', 'PRTF', 'VERB', 'INFN'}
            pos_connects[(second, 'ADVB <- {}'.format('|'.join(POS_group)))] = score(first, 'ADVB')*score(second, *POS_group)
        
        return sorted(pos_connects.items(), key = lambda item: item[1], reverse = True)
        
parser = BigramParser()
print_part = '\n'.join('{} + {}'.format(item[0], ', '.join(item[1])) for item in list(parser.prep_cases.items())[:20])
print('preposition to cases:\n{}\n...\n\nverbal nouns:\n{}'.format(print_part, ', '.join(parser.verbal_nouns)))

preposition to cases:
под + ablt, accs
в течение + gent
благодаря + datv
подле + gent
насупротив + gent
по мере + gent
против + gent
сзади + gent
из-за + gent
смотря по + datv
на + loct, loc2, accs
впереди + gent
надо + ablt
на глазах у + gent
наперекор + datv
из + gent
изо + gent
ради + gent
меж + gent, ablt
по сравнению с + ablt
...

verbal nouns:
скидка, загар, отток, рукоприкладство, тяжба, сдвиг, задержка, ходьба, ловля, пересмотр, воображала, отсрочка, налёт, грызня, ставка, отправка, обжимка, огрызок, беготня, убежище, помолвка, заход, вышивка, выписка, подарок, варка, выход, битва


### Демонстрация работы:

In [13]:
bigramms = ['приветливый взор', 'открытый взору', 'в город', 'освобождение от', 
            'перевозка грузов', 'ложку сахару', 'перевозит грузы', 'умеет плавать', 
            'умение плавать', 'готовый помочь', 'очень хорошо', 'весьма интересный', 
            'быстро бежит', 'пять машин']
for b in bigramms:
    print('* {}\n{}'.format(b, '\n'.join(map(lambda item: '{} : {}'.format(*item), parser.connect(*re.split(' ', b))))))

* приветливый взор
('взор', 'ADJF <- NOUN') : 1.0
* открытый взору
('открытый', 'PRTF -> NOUN') : 0.5
* в город
('в', 'PREP -> NOUN') : 0.999763000236
* освобождение от
('освобождение', 'NOUN -> PREP') : 0.9999990000000001
* перевозка грузов
('перевозка', 'NOUN -> NOUN') : 1.0
* ложку сахару
('ложку', 'NOUN -> NOUN') : 0.3333333333333333
* перевозит грузы
('перевозит', 'VERB -> NOUN') : 0.5
* умеет плавать
('умеет', 'VERB -> INFN') : 1.0
* умение плавать
('умение', 'NOUN -> INFN') : 1.0
* готовый помочь
('готовый', 'ADJF -> INFN') : 0.975903
* очень хорошо
('хорошо', 'ADVB <- ADVB') : 0.5
* весьма интересный
('интересный', 'ADVB <- ADJF') : 0.999999
* быстро бежит
('бежит', 'ADVB <- VERB') : 0.954545
* пять машин
('машин', 'NUMR <- NOUN') : 0.999999


##### Неоднозначные ситуации и вероятности для каждого из вариантов связи:

In [14]:
bigramms = ['готовая помочь', 'три машины', 'мой какаду']
for b in bigramms:
    print('* {}\n{}'.format(b, '\n'.join(map(lambda item: '{} : {}'.format(*item), parser.connect(*re.split(' ', b))))))

* готовая помочь
('готовая', 'ADJF -> INFN') : 0.975903
('помочь', 'ADJF <- NOUN') : 0.012048
* три машины
('машины', 'NUMR <- NOUN') : 0.857139428574
('три', 'VERB -> NOUN') : 0.0037592819550000007
* мой какаду
('какаду', 'ADJF <- NOUN') : 0.1111111111111111
('мой', 'VERB -> NOUN') : 0.05555555555555555


##### Работа написанного нами синтаксического анализатора пары слов для различных типов связи:

In [15]:
bigramms = {'прямообъектный тип связи' : ['уделить внимание', 'вижу лес'],
            'определительный тип связи' : ['очень хорошо', 'важные вопросы', 'актовому залу', 'вполне приемлимо'],
            'отпредложный тип связи' : ['в здание', 'с маслом', 'на полу'],
            'предикат и субъект' : ['спасатели обнаружили'],
            'посессивный тип связи' : ['книга врача', 'жертвы теракта'],
            'аппозитивный тип связи' : ['мальчик Петя'],
            'количественный тип связи' : ['пять машин'],
            'обстоятельственный тип связи' : ['быстро бежать', 'идти медленно']}
for key, value in bigramms.items():
    print('{}:'.format(key))
    for b in value:
        print('* {}\n{}'.format(b, '\n'.join(map(lambda item: '{} : {}'.format(*item), parser.connect(*re.split(' ', b))))))
    print()

количественный тип связи:
* пять машин
('машин', 'NUMR <- NOUN') : 0.999999

отпредложный тип связи:
* в здание
('в', 'PREP -> NOUN') : 0.9997630002360001
* с маслом
('с', 'PREP -> NOUN') : 0.998363
* на полу
('на', 'PREP -> NOUN') : 0.99931

определительный тип связи:
* очень хорошо
('хорошо', 'ADVB <- ADVB') : 0.5
* важные вопросы
('вопросы', 'ADJF <- NOUN') : 0.9999990000000001
* актовому залу
('залу', 'ADJF <- NOUN') : 0.6666666666666666
* вполне приемлимо
('приемлимо', 'ADVB <- ADVB') : 0.05128205128205128

посессивный тип связи:
* книга врача
('книга', 'NOUN -> NOUN') : 0.666666
* жертвы теракта
('жертвы', 'NOUN -> NOUN') : 1.0

предикат и субъект:
* спасатели обнаружили


прямообъектный тип связи:
* уделить внимание
('уделить', 'INFN -> NOUN') : 0.875
* вижу лес
('вижу', 'VERB -> NOUN') : 0.25

аппозитивный тип связи:
* мальчик Петя


обстоятельственный тип связи:
* быстро бежать
('бежать', 'ADVB <- INFN') : 0.954545
* идти медленно




Как можно видеть, наиболее распространенные типы связей, кроме аппозитивного и связи предиката и субъекта, распознаются.

# Часть 3. Поиск синтаксических связей в предложении.

Создадим класс <b>SentParser</b> (унаследованный от <b>BigramParser</b>), который реализует поиск синтаксических связей в предложении при помощи метода parse, который ставит в соответствие предложению упорядоченную последовательность наиболее вероятных связей вида  
<b>[{'nums' : порядковые номера слов данной связи в предложении,   
'words' : слова данной связи,  
'main_w' : главное слово данной связи,  
'label' : тип данной связи,  
'score' : вероятность данной связи}]</b>  
Последовательность упорядочена по параметру 'nums'; каждой связи таким образом ставится в соответствие ровно один вариант правила - он выбирается на основе параметра 'score' (как наиболее вероятный).

У класса <b>SentParser</b> есть два параметра, которые непосредственно влияют на его работу:
* параметр <i>min_score</i> показывает, насколько малой может быть вероятность найденного типа связи для включения ее в список; используется в основном методе parse;
* параметр <i>distance</i> показывает, насколько далеки могут быть слова, которые проверяются нами на синтаксическую связанность; используется во вспомогательном методе neighbors.

Вспомогательный метод neighbors для слова возвращает список "соседних" с ним слов: так как наши правила применялись для упорядоченных слов, то и "соседними" словами будут являться те, что стоят с правой стороны и являются "вторыми" для пары (слово, соседнее с ним слово); как было сказано выше, параметр <i>distance</i> оценивает, насколько далекими могут быть "соседние слова".

Вспомогательный метод word_split выполняет простейшее разбиение на слова, понимая под ними последовательность букв, цифр, нижнего подчеркивания и дефиса: дефис значим, поскольку используемый нами морфоанализатор умеет анализировать такие слова, как "из-за" и "как-то".

In [16]:
print(word_parse('из-за'))
print(word_parse('как-то'))

'из-за':
из-за + (PREP)
'как-то':
как-то + (ADVB)


Таким образом, метод parse класса <b>SentParser</b>, инициализированного с параметром distance = 1, для словосочетания 'x y z w' будет проверять на синтаксическую связанность пары (x, y, (y, z) и (z, w); если параметр distance = 2, то будут проверяться пары (x, y), (x, z), (y, z), (y, w), (z, w) - и именно в такой последовательности.

In [17]:
class SentParser(BigramParser):
    def __init__(self, distance = 2, min_score = 1e-2):
        # bi_parser initialization
        super().__init__()
        # word segmentation function
        self.word_split = lambda text: re.findall('[\w-]+', text)
        
        self.distance = distance
        self.neighbors = lambda sent, num: list(enumerate(sent))[num + 1 : num + self.distance + 1]
        
        self.min_score = min_score
    
    def parse(self, sentence):
        sent = self.word_split(sentence)
        
        sent_parse = [(((i, j), (w_i, w_j)), self.connect(w_i, w_j))
                      for i, w_i in enumerate(sent) for j, w_j in self.neighbors(sent, i)]
        sent_parse = [(pair, parse[0]) for pair, parse in sent_parse if parse and parse[0][1] >= self.min_score]
        connect_dict = lambda b: {'nums' : b[0][0], 'words' : b[0][1],
                                  'main_w' : b[1][0][0], 'label' : b[1][0][1],
                                  'score' : b[1][1]}
        return [connect_dict(connect) for connect in sent_parse]

### Демонстрация работы:

In [18]:
parser = SentParser(1)
trigramms = ['исключительно интересный фильм', 'красный полосатый мяч',
             'любит весело играть', 'при большом желании']
str_connect = lambda c: ', '.join(map(lambda i: '{} : {}'.format(*i), sorted(c.items(), reverse = True)))
for t in trigramms:
    print('* {}\n{}'.format(t, '\n'.join(map(str_connect, parser.parse(t)))))

* исключительно интересный фильм
words : ('исключительно', 'интересный'), score : 0.24999975, nums : (0, 1), main_w : интересный, label : ADVB <- ADJF
words : ('интересный', 'фильм'), score : 0.999999, nums : (1, 2), main_w : фильм, label : ADJF <- NOUN
* красный полосатый мяч
words : ('полосатый', 'мяч'), score : 0.999999, nums : (1, 2), main_w : мяч, label : ADJF <- NOUN
* любит весело играть
words : ('весело', 'играть'), score : 0.857142, nums : (1, 2), main_w : играть, label : ADVB <- INFN
* при большом желании
words : ('большом', 'желании'), score : 0.272727, nums : (1, 2), main_w : желании, label : ADJF <- NOUN


In [19]:
parser = SentParser(2)
for t in trigramms:
    print('* {}\n{}'.format(t, '\n'.join(map(str_connect, parser.parse(t)))))

* исключительно интересный фильм
words : ('исключительно', 'интересный'), score : 0.24999975, nums : (0, 1), main_w : интересный, label : ADVB <- ADJF
words : ('интересный', 'фильм'), score : 0.999999, nums : (1, 2), main_w : фильм, label : ADJF <- NOUN
* красный полосатый мяч
words : ('красный', 'мяч'), score : 0.666666, nums : (0, 2), main_w : мяч, label : ADJF <- NOUN
words : ('полосатый', 'мяч'), score : 0.999999, nums : (1, 2), main_w : мяч, label : ADJF <- NOUN
* любит весело играть
words : ('любит', 'играть'), score : 1.0, nums : (0, 2), main_w : любит, label : VERB -> INFN
words : ('весело', 'играть'), score : 0.857142, nums : (1, 2), main_w : играть, label : ADVB <- INFN
* при большом желании
words : ('при', 'желании'), score : 0.99931, nums : (0, 2), main_w : при, label : PREP -> NOUN
words : ('большом', 'желании'), score : 0.272727, nums : (1, 2), main_w : желании, label : ADJF <- NOUN


Как можно видеть, равенство параметра distance двум позволяет находить все возможные синтаксические связи в триграммах: при distance = 1 обрабатываются только соседние слова и не учитываются более сложные, комбинированные типы связей (кроме последовательных, как в случае с "исключительно интересным фильмом").

###### Тем не менее, большое значение параметра distance может приводить к ошибочно найденным связям:

In [20]:
parser = SentParser(3)
trigramms = ['очень быстро бежать', 'связь морфологии и синтаксиса']
for t in trigramms:
    print('* {}\n{}'.format(t, '\n'.join(map(str_connect, parser.parse(t)))))

* очень быстро бежать
words : ('очень', 'быстро'), score : 0.954545, nums : (0, 1), main_w : быстро, label : ADVB <- ADVB
words : ('очень', 'бежать'), score : 1.0, nums : (0, 2), main_w : бежать, label : ADVB <- INFN
words : ('быстро', 'бежать'), score : 0.954545, nums : (1, 2), main_w : бежать, label : ADVB <- INFN
* связь морфологии и синтаксиса
words : ('связь', 'морфологии'), score : 0.333333, nums : (0, 1), main_w : связь, label : NOUN -> NOUN
words : ('связь', 'синтаксиса'), score : 1.0, nums : (0, 3), main_w : связь, label : NOUN -> NOUN
words : ('морфологии', 'синтаксиса'), score : 0.999997, nums : (1, 3), main_w : морфологии, label : NOUN -> NOUN


Обратим отдельное внимание на граф зависимостей обрабатываемых словосочетаний: в случае distance = 1 словосочетание разбивается на некоторое количество деревьев: используемые нами правила не предполагают ситуации, в которой одно из слов становится зависимым от слова слева от него и слова справа.

##### Но уже при distance = 2 возможно появление циклов:

In [21]:
parser = SentParser(2)
t = 'начавшееся днем кино'
print('* {}\n{}'.format(t, '\n'.join(map(str_connect, parser.parse(t)))))

* начавшееся днем кино
words : ('начавшееся', 'днем'), score : 0.5, nums : (0, 1), main_w : начавшееся, label : PRTF -> NOUN
words : ('начавшееся', 'кино'), score : 0.333333, nums : (0, 2), main_w : кино, label : PRTF <- NOUN
words : ('днем', 'кино'), score : 0.0714285, nums : (1, 2), main_w : днем, label : NOUN -> NOUN


* начавшееся -> днем, днем -> кино, кино -> начавшееся  
появление цикла по причине реализации правила NOUN -> NOUN (+ gen, gen_2)

Лишнюю связь можно было бы отбросить, но для этого нужно работать с построенным графом целиком и строить правила для согласованности правил синтаксических связей, а также оцениванивать "вероятность" построенных деревьев: действительно, проблема данного словосочетания и в его омонимичности, и лишней будет другая связь:

In [22]:
t = 'мероприятие, начавшееся "Днем Кино"'
print('* {}\n{}'.format(t, '\n'.join(map(str_connect, parser.parse(t)))))

* мероприятие, начавшееся "Днем Кино"
words : ('начавшееся', 'Днем'), score : 0.5, nums : (1, 2), main_w : начавшееся, label : PRTF -> NOUN
words : ('начавшееся', 'Кино'), score : 0.333333, nums : (1, 3), main_w : Кино, label : PRTF <- NOUN
words : ('Днем', 'Кино'), score : 0.0714285, nums : (2, 3), main_w : Днем, label : NOUN -> NOUN


Этот пример частично освещает разные стороны проблемы построения деревьев зависимости, следования правилам и проч. В данном задании они не будут решаться, потому что в большинстве случаев (как будет видно далее) мы будем получать удовлетворительный результат; тем не менее, будем помнить о возможности получения цикла при distance > 1.

# Часть 4. Синтаксический анализ текста (частичный парсер).

Проанализируем работу нашего класса <b>SentParser</b> для нескольких предложений первого текста, сегментированного классом <b>Split</b>, продемонстрировав работу синтаксического анализатора не на словосочетаниях, а на реальных предложениях:

In [23]:
for i, sent_i in split[(0,1)][:2]:
    print('* {}\n{}'.format(sent_i.replace('\n', ' '), '\n'.join(map(str_connect, parser.parse(sent_i)))))

* Я  самым внимательным образом изучил карты города, но так и  не отыскал на  них  улицу  д'Осейль
words : ('самым', 'образом'), score : 0.148148, nums : (1, 3), main_w : образом, label : ADJF <- NOUN
words : ('внимательным', 'образом'), score : 0.3333333333333333, nums : (2, 3), main_w : образом, label : ADJF <- NOUN
words : ('образом', 'карты'), score : 0.90909, nums : (3, 5), main_w : образом, label : NOUN -> NOUN
words : ('изучил', 'карты'), score : 0.045454, nums : (4, 5), main_w : изучил, label : VERB -> NOUN
words : ('карты', 'города'), score : 0.9899470201019999, nums : (5, 6), main_w : карты, label : NOUN -> NOUN
words : ('на', 'улицу'), score : 0.99931, nums : (12, 14), main_w : на, label : PREP -> NOUN
* Надо сказать, что  я  рылся отнюдь  не  только  в современных  картах,  поскольку мне  было  известно, что  подобные  названия нередко меняются
words : ('что', 'рылся'), score : 0.014925, nums : (2, 4), main_w : рылся, label : ADVB <- VERB
words : ('в', 'картах'), score : 0.

Словосочетание "отыскал на них" вообще не обработалось! Кроме того, не была выделена связь "самым внимательным" - поскольку "самый" является местоименным прилагательным, но не наречием; невыделение прочих связей, таких как "образом изучил" (обратный порядок слов - не рассматриваем), "улицу д'Осейль" (аппозитивный тип связи - не рассматриваем) обсуждалось ранее.

Ранее мы уже говорили об особенностях разбора местоимений, - с одной стороны, зачастую в их разборах присутствует отличная от NPRO часть речи, что позволяет нам применить имеющиеся правила; с другой стороны, в их разборах никогда не присутствует существительных. Мы не будем переписывать все правила таким образом, чтобы они работали и для местоимений, заменяющих существительные; вместо этого мы добавим правило связи предлога и местоимения по аналогии со связью Prep -> N: в нем будет проверяться, есть ли вообще падеж у предлога + проверим, что это местоимение не может играть роль прилагательного или числительного, то есть во всех разборах оно обладает тегом 'NPRO'.

In [24]:
print(parser.morph_tags('них', 'POS'))
print(parser.morph_tags('кого-то', 'POS'))

{'NPRO'}
{'NPRO'}


Поскольку в наших правилах есть 'N -> Prep', где Ν - отглагольное существительное, то было бы логичным добавить связь <b>'V -> Prep'</b> (будем помнить о том, что при distance > 1 в словосочетании "получить освобождение от работ" будет найдена лишняя связь; впрочем, и с имеющимся набором правил в словосочетании "поедание хлеба с маслом" будет найдена лишняя связь - более того, неясно, "с маслом" - это определение хлеба или определение поедания: все не так просто).

In [25]:
t = 'поедание хлеба с маслом'
print('* {}\n{}'.format(t, '\n'.join(map(str_connect, parser.parse(t)))))

* поедание хлеба с маслом
words : ('поедание', 'хлеба'), score : 0.923076, nums : (0, 1), main_w : поедание, label : NOUN -> NOUN
words : ('поедание', 'с'), score : 0.998363, nums : (0, 2), main_w : поедание, label : NOUN -> PREP
words : ('с', 'маслом'), score : 0.998363, nums : (2, 3), main_w : с, label : PREP -> NOUN


## Text Parser
Итак, напишем парсер нашего текста. Он реализует свойства как класса <b>Split</b>, так и класса <b>SentParser</b>, - унаследуем от них создаваемый класс <b>Parser</b>.

### Модификации методов родительских классов и методы, их использующие

* Используя метод word_split класса <b>SentParser</b>, добавим метод сегментации на слова: word_segment.
* Дополним метод connect класса <b>BigramParser</b>, унаследованный классом <b>SentParser</b>, двумя правилами: V -> Prep и Prep -> Npro.
* Модифицируем метод parse класса <b>SentParser</b>, написав новый метод sent_parse: по номеру предложения в структуре, полученной методами класса <b>Split</b>, он возвращает список вида <b>[((индекс главного слова, индекс зависимого слова), тип связи)]</b>, где индексирование соответствует индексам слов в структуре, полученной применением метода word_segment.
* О прочих методах и членах класса <b>Parser</b> будет сказано позже.

### Члены класса Parser

Помимо членов, определяемых классами <b>Split</b> и <b>SentParser</b>, определим следующие члены:
* connects - словарь найденных синтаксических связей: он будет инициализироваться, как <b>{(индекс главного слова, индекс зависимого слова) : тип связи}</b> путем применения метода sent_parse для всех предложений структуры.
* graph - граф найденных синтаксических связей, представленный списком смежности зависимых от него слов с информацией о типе связи, то есть словарь вида <b>{индекс главного слова : [{зависимое слово : тип зависимости}]</b>.

Инициализация graph производится методом graph_construct на основе полученного словаря connects.

In [26]:
class Parser(Split, SentParser):
    def __init__(self, *names, distance = 1, min_score = 1e-2):
        Split.__init__(self, *names)
        SentParser.__init__(self, distance, min_score)
        # word segmentation
        self.word_segment = lambda: self.segment(self.texts, self.word_split)
        # dictionary of syntax connections
        self.connects = {}
        # syntax graph
        self.graph = {}
    def connect(self, first, second):
        pos_connects = dict(SentParser.connect(self, first, second))
        
        POSes = dict(zip(('first', 'second'), map(lambda word: set(self.morph_tags(word, 'POS')), (first, second))))
        check_POSes = lambda p_1, p_2: set(p_1.split(','))&POSes['first'] and set(p_2.split(','))&POSes['second']
        score = lambda word, *pos_tags: sum(score for tag, score in self.tag_scores(word).items() for t in pos_tags
                                            if 'None' not in t and set(t.split(',')) in tag)
        
        # {VERB,INFN} -> PREP
        if check_POSes('VERB,INFN', 'PREP'):
            POS_group = POSes['first']&{'VERB', 'INFN'}
            pos_connects[(first, '{} -> PREP'.format('|'.join(POS_group)))] = score(first, *POS_group)*score(second, 'PREP')
        
        # PREP -> NPRO (~ PREP -> NOUN)
        if check_POSes('PREP', 'NPRO') and score(second, 'NPRO') > 1.0 - self.min_score:
            if first.lower() in self.prep_cases and self.prep_cases[first.lower()]&self.morph_tags(second, 'case'):
                pos_connects[(first, 'PREP -> NPRO')] = score(first, 'PREP')*score(second, 'NPRO')
        
        return sorted(pos_connects.items(), key = lambda item: item[1], reverse = True)
    def sent_parse(self, i):
        sent_parse = self.parse(self.texts[i])
        reverse = lambda nums: tuple(i for i in reversed(nums))
        sent_parse = [(reverse(b['nums']) if b['words'].index(b['main_w']) else b['nums'], b['label']) for b in sent_parse]
        return [((i + (b[0][0],), i + (b[0][1],)), b[1]) for b in sent_parse]
    def text_parse(self):
        self.par_segment()
        self.sent_segment()
        self.connects = {connect : label for i, sent_i in self for connect, label in self.sent_parse(i)}
        self.sents = self.texts.copy()
        self.word_segment()
        self.words = self.texts
        self.graph_construct()
        return (sorted(self.sents.items()), sorted(self.words.items()), 
                sorted(self.connects.items()), sorted(self.graph.items()))
    def graph_construct(self):
        for pair, label in self.connects.items():
            if pair[0] not in self.graph:
                self.graph[pair[0]] = [{pair[1] : label}]
            else:
                self.graph[pair[0]].append({pair[1] : label})
        return self.graph
    def graph_repr(self):
        graph_words = {}
        for main_w, connects in self.graph.items():
            nums = sorted((main_w, *map(lambda w_l: list(w_l)[0], connects)))
            num_labels = sorted(map(lambda w_l: list(w_l.items())[0], connects), key = lambda w_l: w_l[0])
            graph_words[main_w] = (tuple(map(lambda w: parser[w], nums)),
                                   tuple(map(lambda w_l: w_l[1], num_labels)), 
                                   self.sents[main_w[:-1]])
        return graph_words

    def tree_repr(self, main_w, depth, words, labels):
        if main_w in self.graph and main_w not in words:
            words.append(main_w)
            for w_l in self.graph[main_w]:
                word, label = list(w_l.items())[0]
                connect = ' '.join(map(lambda w: self[w], sorted([main_w, word])))
                labels.append((depth, connect, label))
                self.tree_repr(word, depth + 1, words, labels)
        else:
            words.append(main_w)
        return tuple(map(lambda w: self[w], sorted(list(set(words))))), labels, self.sents[main_w[:-1]]
    def forest_repr(self):
        return {main_w : self.tree_repr(main_w, 0, [], []) for main_w in self.graph}
    
    def connect_search(self, label):
        l_values = {'concord' : ' <- NOUN', 'AGR' : ' <- NOUN', '<-N' : ' <- NOUN',
                    '->noun' : ' -> NOUN', '->N' : ' -> NOUN',
                    '->infinitive' : ' -> INFN', '->Inf' : ' -> INFN',
                    'infinitive' : 'INFN', 'Inf' : 'INFN',
                    'adverb<-' : 'ADVB <- ', 'Adv<-' : 'ADVB <- ',
                    'adverb' : 'ADVB', 'Adv' : 'ADVB <- ',
                    'preposition' : 'PREP', 'Prep' : 'PREP', 
                    '->Prep' : ' -> PREP', 'Prep->' : 'PREP -> '}
        order = lambda key: (self[key[0]], self[key[1]]) if key[0] < key[1] else (self[key[1]], self[key[0]])
        return [(*order(key), value) for key, value in self.connects.items() 
                if value == label or (label in l_values and l_values[label] in value)]

### Работа модифицированного метода connect:

In [27]:
parser = Parser()
trigramms = ['бегать по лесу', 'ставка на них']
for t in trigramms:
    print('* {}\n{}'.format(t, '\n'.join(map(str_connect, parser.parse(t)))))

* бегать по лесу
words : ('бегать', 'по'), score : 1.0, nums : (0, 1), main_w : бегать, label : INFN -> PREP
words : ('по', 'лесу'), score : 1.0, nums : (1, 2), main_w : по, label : PREP -> NOUN
* ставка на них
words : ('ставка', 'на'), score : 0.99931, nums : (0, 1), main_w : ставка, label : NOUN -> PREP
words : ('на', 'них'), score : 0.99930900069, nums : (1, 2), main_w : на, label : PREP -> NPRO


## Демонстрация работы.
Расскажем про реализованные методы класса <b>Parser</b> и покажем их работу на примере рассказа Говарда Лавкрафта "Музыка Эриха Цанна" (http://lib.ru/INOFANT/LAWKRAFT/muz.txt). Для начала будем рассматривать работу класса с distance = 1.

### Метод text_parse

Данный метод реализует создание структуры уровня предложений и уровня слов на основе текстов, сопоставляя каждому предложению индекс (x, y, z), а слову - индекс (x, y, z, w), где x - номер текста, y - номер абзаца, z - номер предложения, w - номер слова; также создает граф зависимостей на основе найденных в предложениях синтаксических связей.

Его работа происходит в несколько этапов:
* применение метода par_segment - сегментация текстов на абзацы;
* применение метода sent_segment - сегментация полученной структуры на предложения;
* инициализация словаря connects путем применения к каждому предложению полученной структуры метода sent_parse;
* создание копии структуры, соответствующей уровню предложений: член sents;
* применение метода word_segment - сегментация полученной структуры на слова: получение члена words;
* инициализация графа зависимостей graph при помощи метода graph_construct.

Метод возвращает упорядоченные по индексам инициализированные этим методом члены sents, words, connect и graph:

In [28]:
parser = Parser('text_1.txt', distance = 1)
print('\n{}\n'.format('.'*100).join(map(lambda s: '\n'.join(map(lambda c: '{} : {}'.format(*c), s[:10])), parser.text_parse())))

(0, 0) : Говард Ф
(0, 1) : Лавкрафт
(0, 2) : Музыка Эриха Цанна
(1, 0) : Я  самым внимательным образом изучил карты города, но так и  не отыскал
на  них  улицу  д'Осейль
(1, 1) : Надо сказать, что  я  рылся отнюдь  не  только  в
современных  картах,  поскольку мне  было  известно, что  подобные  названия
нередко меняются
(1, 2) : Напротив, я, можно сказать,  по уши залез  в седую старину
и, более того, лично обследовал интересовавший меня район, уже  не  особенно
обращая  внимания на  таблички  и  вывески,  в  поисках  того,  что  хотя бы
отдаленно походило на интересовавшую  меня улицу д'Осейль
(1, 3) : Однако,  несмотря
на  все мои усилия, вынужден сейчас не без  стыда признаться,  что  так и не
смог отыскать нужные мне дом, улицу, и даже приблизительно определить район,
где,  в  течение  последних месяцев  моей  обездоленной  жизни,  я,  студент
факультета метафизики, слушал музыку Эриха Занна
(2, 0) : Меня отнюдь не  удивляет  подобный провал в памяти, поскольку за период
жизни на 

### Метод connect_search

Данный метод возвращает список пар слов (в их изначальном порядке), которым методом text_parse был сопоставлен подаваемый на вход тип связи.

Помимо обработки прямого совпадения типа связи реализован пользовательский словарь:

In [29]:
for bond in ['concord', 'PRTF <- NOUN', '->noun', 'Prep->', 'Inf', 'Adv<-']:
    print('* connections with {}:\n{}\n'.format(bond, '\n'.join(map(lambda c: '{} {} : {}'.format(*c),
                                               parser.connect_search(bond)[:10]))))

* connections with concord:
Дорожное покрытие : ADJF <- NOUN
бешеных мотивах : ADJF <- NOUN
необитаемого помещения : ADJF <- NOUN
его голове : ADJF <- NOUN
мой плащ : ADJF <- NOUN
потертой одежде : PRTF|ADJF <- NOUN
колдовские мотивы : ADJF <- NOUN
этой мансарде : ADJF <- NOUN
странный тип : ADJF <- NOUN
их источника : ADJF <- NOUN

* connections with PRTF <- NOUN:
обездоленной жизни : PRTF <- NOUN
запомнившихся мелодий : PRTF <- NOUN
зашторенного окна : PRTF <- NOUN
скрюченный мужчина : PRTF <- NOUN
покосившимися домами : PRTF <- NOUN
запертой двери : PRTF <- NOUN
захламленному столу : PRTF <- NOUN
одуряющими звуками : PRTF <- NOUN
зашторенного окна : PRTF <- NOUN
разбросанные груды : PRTF <- NOUN

* connections with ->noun:
булыжником улицы : NOUN -> NOUN
вылетел конец : VERB -> NOUN
в получасе : PREP -> NOUN
на полу : PREP -> NOUN
под дверью : PREP -> NOUN
игру Эриха : NOUN -> NOUN
на земле : PREP -> NOUN
к столу : PREP -> NOUN
юбку матери : NOUN -> NOUN
скрипнул ставень : VERB -> N

Можем видеть, что правила синтаксических связей достаточно неплохо работают!  
По крайней мере, вырванные из контекста, данные словосочетания выглядят абсолютно нормально.

### Методы представления graph

Нами было реализовано два метода представления полученного списка смежностей:

#### graph_repr: 

* возвращает словарь вида <b>{главное слово : (упорядоченный список из зависимых слов и главного, список типов связей, предложение с этими словами)]}</b></b>, где упорядочивание производится согласно индексам слов, то есть в том порядке, в котором они изначально были в предложении;  
* так, словосочетание "срочная перевозка грузов" при distance = 1 будет обработано следующим образом:
перевозка : срочная перевозка грузов : A <- N, N -> N

Покажем работу этого метода, отсортировав полученный словарь по длине списка слов:

In [30]:
graph_repr = parser.graph_repr()
for main_w, bond_words in list(sorted(list(graph_repr.items()),
                                      key = lambda item: len(item[1][0]),
                                      reverse = True))[:20]:
    print('{} : {} : {}'.format(parser[main_w], ' '.join(bond_words[0]), ', '.join(bond_words[1])))

покрытие : Дорожное покрытие улицы : ADJF <- NOUN, NOUN -> NOUN
места : этого места сосредоточения : ADJF <- NOUN, NOUN -> NOUN
ушел : совсем ушел в : ADVB <- VERB, VERB -> PREP
светом : лунным светом крыш : ADJF <- NOUN, NOUN -> NOUN
бездне : невообразимой бездне мрака : ADJF <- NOUN, NOUN -> NOUN
желание : давнее желание выглянуть : ADJF <- NOUN, NOUN -> INFN
крыши : поблескивающие крыши домов : PRTF <- NOUN, NOUN -> NOUN
желание : своенравное желание выглянуть : ADJF <- NOUN, NOUN -> INFN
время : любое время суток : ADJF <- NOUN, NOUN -> NOUN
помигивали : приветливо помигивали огни : ADVB <- VERB, VERB -> NOUN
порыв : налетевший порыв ветра : PRTF <- NOUN, NOUN -> NOUN
превышавшей : намного превышавшей уровень : ADVB <- PRTF, PRTF -> NOUN
расположение : дружеское расположение в : ADJF <- NOUN, NOUN -> PREP
захлопали : оглушительно захлопали по : ADVB <- VERB, VERB -> PREP
обилие : повсеместное обилие пыли : ADJF <- NOUN, NOUN -> NOUN
вздрогнул : невольно вздрогнул от : ADVB <- VERB,

Поскольку distance = 1, то длина полученных наиболее длинных словосочетаний не превышает 3: поскольку список смежности для каждого из слов состоит не более чем из двух зависимых от него.

Тем не менее, это первый шаг к выделению словосочетаний, полученных применением нескольких правил.

#### forest_repr:

* вспомогательный метод <i>tree_repr</i>:  
    * для одного из "главных" слов рекурсивно получает все индексы слов, до которых можно добраться, следуя стрелкам найденных синтаксических связей, а также список триплетов (глубина рекурсии, словосочетание, тип связи); затем упорядочивает индексы и сопоставляет им сами слова, полученный же список пар остается в порядке действия рекурсии; возвращает отсортированный список слов и список триплетов (глубина рекурсии, словосочетание, тип связи) в рекурсивном порядке;  
    * так, словосочетание "срочная перевозка грузов до города" для слова "перевозка" при distance = 1 будет обработано следующим образом:  
срочная перевозка грузов до города:
...срочная перевозка : Α <- N
перевозка грузов : N -> N
грузов до : N -> Prep
до города : Prep -> N  
* <b>forest_repr</b>:
    * возвращает словарь вида <b>{главное слово : дерево, полученное применением метода tree_repr}</b>
    * стоит отметить, что многие из полученных последовательностей слов будут вложенными друг в друга, так, словосочетание "срочная перевозка грузов до города" для слова "грузов" дает дерево "грузов до города" $\subset$ "срочная перевозка грузов до города"
    
Покажем работу этого метода, отсортировав полученный словарь по длине списка слов:

In [31]:
forest_repr = parser.forest_repr()
for main_w, tree_words in list(sorted(list(forest_repr.items()),
                                      key = lambda tree: len(tree[1][0]),
                                      reverse = True))[:10]:
    print('{}:\n{}\n'.format(' '.join(tree_words[0]), 
                           '\n'.join(map(lambda trio: '    {}{} : {}'.format(trio[0] * '...', *trio[1:]), 
                                         tree_words[1]))))

после чего снова опустился на стул:
    опустился на : VERB -> PREP
    ...на стул : PREP -> NOUN
    снова опустился : ADVB <- VERB
    ...чего снова : ADVB <- ADVB
    ......после чего : ADVB <- ADVB

буквально светилось от облегчения при виде:
    светилось от : VERB -> PREP
    ...от облегчения : PREP -> NOUN
    ......облегчения при : NOUN -> PREP
    .........при виде : PREP -> NOUN
    буквально светилось : ADVB <- VERB

этого места сосредоточения ночных кошмаров:
    этого места : ADJF <- NOUN
    места сосредоточения : NOUN -> NOUN
    ...сосредоточения ночных : NOUN -> NOUN
    ......ночных кошмаров : NOUN -> NOUN

давнее желание выглянуть из того:
    желание выглянуть : NOUN -> INFN
    ...выглянуть из : INFN -> PREP
    ......из того : PREP -> NOUN
    давнее желание : ADJF <- NOUN

в получасе ходьбы от университета:
    в получасе : PREP -> NOUN
    ...получасе ходьбы : NOUN -> NOUN
    ......ходьбы от : NOUN -> PREP
    .........от университета : PREP -> NOUN

оглушитель

Можем видеть, что словосочетания стали гораздо сложнее: вместе с тем, они не потеряли осмысленности. Отметим тот факт, что в большинстве выведенных нами случаев вершиной дерева является глагол, что соответствует идее грамматики зависимостей, где вершиной предложения признается глагольная форма.

Исключения составляют те словосочетания, вершина которых находится на расстоянии больше, чем 1, от слова, от которого они зависимы: например, от глагола. Поэтому следующим естественным шагом является увеличение параметра distance.

### Увеличение параметра distance.

In [32]:
parser = Parser('text_1.txt', distance = 2)
parser.text_parse()
graph_repr = parser.graph_repr()
forest_repr = parser.forest_repr()

Посмотрим, как выглядят списки смежности в нашем графе (то есть слова, зависимые от одного):

In [33]:
for main_w, bond_words in list(sorted(list(graph_repr.items()),
                                      key = lambda item: len(item[1][0]),
                                      reverse = True))[:10]:
    print('{} : {} : {}'.format(parser[main_w], ' '.join(bond_words[0]), ', '.join(bond_words[1])))

желание : свое давнее желание выглянуть из : ADJF <- NOUN, ADJF <- NOUN, NOUN -> INFN, NOUN -> PREP
опустился : чего снова опустился на стул : ADVB <- VERB, ADVB <- VERB, VERB -> PREP, VERB -> NOUN
постучал : несколько раз постучал в дверь : ADVB <- VERB, ADVB <- VERB, VERB -> PREP, VERB -> NOUN
доковылял : после чего доковылял до двери : ADVB <- VERB, ADVB <- VERB, VERB -> PREP, VERB -> NOUN
душу : мучившую его душу тайну : PRTF <- NOUN, ADJF <- NOUN, VERB -> NOUN
желание : своенравное желание выглянуть из : ADJF <- NOUN, NOUN -> INFN, NOUN -> PREP
свечу : еще одну свечу в : ADVB <- VERB, ADJF <- NOUN, VERB -> PREP
превышавшей : намного превышавшей уровень крыш : ADVB <- PRTF, PRTF -> NOUN, PRTF -> NOUN
привести : отчаянно привести старика в : ADVB <- INFN, INFN -> NOUN, INFN -> PREP
находилось : все что находилось в : ADVB <- VERB, ADVB <- VERB, VERB -> PREP


При увеличении distance мы получаем не самый верный граф зависимостей, это касается сочетания существительных и предлогов: подразумевается, что их связь неразрывна (в отличие от того же согласования или управление глагола существительным). Мы не будем переписывать наш класс, поскольку исключения касаются не только правил N -> Prep и Prep -> N, и это как раз и является правилами для использования правил синтаксических связей, которые мы и не собирались вырабатывать.

##### Посмотрим теперь, как строятся деревья.

С увеличением параметра distance возможны связи, которые неверны в контексте (при distance = 1 мы можем быть уверены в том, что эти слова были соседними в исходном тексте): поэтому будем выводить предложения, содержащие найденные деревья.

In [34]:
for main_w, tree_words in list(sorted(list(forest_repr.items()),
                                      key = lambda tree: len(tree[1][0]),
                                      reverse = True))[:5]:
    print('{}\n\n{}:\n{}\n'.format(tree_words[2], ' '.join(tree_words[0]), 
                           '\n'.join(map(lambda trio: '    {}{} : {}'.format(trio[0] * '...', *trio[1:]), 
                                         tree_words[1]))))

По массивному, сложенному из  черного камня мосту  улица эта пересекала
темную реку, вдоль которой располагались кирпичные стены складских помещений
с  помутневшими  окнами

вдоль располагались кирпичные стены складских помещений с помутневшими окнами:
    располагались стены : VERB -> NOUN
    ...кирпичные стены : ADJF <- NOUN
    ...стены помещений : NOUN -> NOUN
    ......помещений с : NOUN -> PREP
    .........с окнами : PREP -> NOUN
    ............помутневшими окнами : PRTF <- NOUN
    ......складских помещений : ADJF <- NOUN
    вдоль располагались : ADVB <- VERB

Изредка
попадались и такие места, где как бы падавшие друг другу навстречу дома чуть
ли не  смыкались своими крышами, образуя некое  подобие арки

где как падавшие друг другу навстречу дома ли:
    падавшие друг : PRTF -> NOUN
    как падавшие : ADVB <- PRTF
    ...где как : ADVB <- ADVB
    падавшие другу : PRTF -> NOUN
    ...другу дома : NOUN -> NOUN
    ......навстречу дома : ADVB <- ADVB
    ......дома ли : NOUN -

Далеко не все деревья представляют собой правильный разбор: сказывается то, что не любая пара слов, разделенная словом, может вообще анализиоваться нашими правилам. Тем не менее, первое словосочетание представляет собой пример очень хорошего разбора:

<b><i>вдоль располагались кирпичные стены складских помещений с помутневшими окнами</i></b>

данное словосочетание не только было разобрано правильно, но еще и практически полностью совпадает с частью исходного сложноподчиненного предложения (с исключенным из него словом "которой", связывающим эту часть с основной).

На самом деле, связь "располагались стены" случайна, поскольку у нас нет правила, устанавливающего связь предиката и субъекта, а у слова "стены" совпадают формы именительного и винительного падежа; несмотря на это, разбор не потерял от этого своей правильности.

Заметим также, что несмотря на то, что мы не пытались разбивать предложения на составные части по запятым, найденные словосочетания все равно находятся в одной из частей предложений; таким образом, построенные деревья для сложного предложения соответствуют его частям.

Есть все основания полагать, что на более простых текстах (менее литературных) построенный нами парсер будет выделять правильные словосоччетания с их правильным разбором.